In [ ]:
import numpy as np

In [ ]:
sys.path.append('/home/users/zhiru/linkage-equillibrium/code')
import parameters
import config

params = parameters.params
regime = 'r'
param_idx = 21
N = params[regime][param_idx][2]
r = params[regime][param_idx][6]

In [ ]:
# NR = 1e5
# r = 0.1

In [ ]:
# load the cached frequency trajectories
# (output from simulations/run_simulate_twolocus.sh and simulations/gz_freq_arr_to_npy.py)
save_path = '/home/groups/bhgood/LE_simulations_out/cached_freqs'
all_fs = np.load(os.path.join(save_path, 'fs_%s_%d.npy'%(regime,param_idx)))

f11s = all_fs[:, 0]
f10s = all_fs[:, 1]
f01s = all_fs[:, 2]
f00s = 1-f11s-f10s-f01s
fAs = f11s+f10s
fBs = f11s+f01s

In [ ]:
# downsample the trajectories, otherwise multinomial sampling takes too long to run
num_downsampled = int(1e7)
rows = np.random.randint(all_fs.shape[0], size=num_downsampled)
downsampled_fs = all_fs[rows, :]

In [ ]:
_f00s = 1 - downsampled_fs.sum(axis=1)
sample_probs = np.hstack([downsampled_fs, _f00s.reshape((-1, 1))])
sample_probs = np.around(sample_probs, 6)

In [ ]:
%%time
n = 10000  # note this n goes into the theory curve
num_points = sample_probs.shape[0]
samples = np.zeros((num_points, 4))
for i in range(num_points):
    samples[i] = np.random.multinomial(n, sample_probs[i])

In [ ]:
flims = np.logspace(-3, -1, 20)
nonzero_probs = []
nAs = samples[:, 0] + samples[:, 1]
nBs = samples[:, 0] + samples[:, 2]
for i, flim in enumerate(flims):
    # choose a narrow frequency window around flim (f0)
    fmin = flim * 0.9
    fmax = flim * 1.1
    nmin = fmin * n
    nmax = fmax * n
    
    mask = (nAs >= nmin) & (nAs <= nmax) & (nBs >= nmin) & (nBs <= nmax)
    prob = np.mean(np.all(samples[mask, :]>0, axis=1))
    nonzero_probs.append(prob)

In [ ]:
np.save('f0s_sim', flims)
np.save('4g_prob_sim', nonzero_probs)